In [1]:
import numpy as np

<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">QR factorization</h1>

Let $A\in\mathbb{R}^{n\times m},\ n\geq m$ be a rectangular matrix. A possible decomposition is given by $A = QR$, with $Q\in\mathbb{R}^{n\times n}$ orthogonal and $R\in\mathbb{R}^{n\times m}$ upper triangular.

<span style="color:orange">N.B.</span> Economy QR factorization: $A = Q_1R_1$ with $Q\in\mathbb{R}^{n\times m}$ and $R\in\mathbb{R}^{m\times m}$

<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">Gram-Schmidt orthogonalization</h1>


<span style="color:red">Condition:</span> The columns of $A$ must be linearly independent ($A$ should be a full rank matrix)

### Algorithm

In [2]:
def Gram_Schmidt_orthogonalization(A):
    n = A.shape[0]
    m = A.shape[1]
    
    ### R ###
    R = np.zeros((m, m))
    
    ### Q ###  
    Q = np.zeros((n, m))
    
    Q[:, 0] = A[:, 0] / np.linalg.norm(A[:, 0])
    R[0][0] = np.linalg.norm(A[:, 0])
    
    for i in range(1, m):
        s = 0
        for j in range(i):
            s += np.dot(Q[:, j], np.dot(Q[:, j].T, A[:, i]))
        
        q_i = A[:, i] - s
        norm = np.linalg.norm(q_i)
        Q[:, i] = q_i / norm
        
        R[i, i] = norm
        
    for i in range(m):
        for j in range(i+1, m):
            R[i, j] = np.dot(Q[:, i].T, A[:, j])
        
    return Q, R

In [3]:
A = np.random.randint(1, 10, (5, 5))

print(f'A = \n{A}\n')

Q, R = Gram_Schmidt_orthogonalization(A)

print(f'Q = \n{Q}\n')
print(f'R = \n{R}\n')
print(f'QR = \n{np.dot(Q, R)}\n')

A = 
[[4 6 8 6 9]
 [6 9 5 6 2]
 [3 8 7 9 4]
 [3 6 3 6 7]
 [1 5 4 9 8]]

Q = 
[[ 0.47471266 -0.2225657   0.80244291  0.23878747  0.15549378]
 [ 0.71206899 -0.33384855 -0.42619499 -0.00344074 -0.44704461]
 [ 0.3560345   0.58047541  0.16629082 -0.71292167 -0.01943672]
 [ 0.3560345   0.15338988 -0.38312686  0.19107585  0.81634234]
 [ 0.11867817  0.69175826 -0.00209359  0.63103202 -0.33042428]]

R = 
[[ 8.42614977 14.83477073 11.39310392 13.5293109  10.56235676]
 [ 0.          4.68290268  3.84076218  9.0319568   6.25890846]
 [ 0.          0.          4.29484912  1.43650142  4.33612274]
 [ 0.          0.          0.          1.82152864  5.67630621]
 [ 0.          0.          0.          0.          3.49861001]]

QR = 
[[4. 6. 8. 6. 9.]
 [6. 9. 5. 6. 2.]
 [3. 8. 7. 9. 4.]
 [3. 6. 3. 6. 7.]
 [1. 5. 4. 9. 8.]]



<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">Householder matrix reflections</h1>

The Householder reflection matrix is defined as $$P =I -\beta v v^T,\ \ \beta = \frac{2}{||v||_2^2}$$

Then when applied to a vector we get the annihilation of all terms of the vector except for the first: 

$$Px = \alpha e_1$$

This property holds if we defined the vector $v$ as $$ v = x -\alpha e_1,\ \ \ \alpha = \pm ||x||\theta$$

### Algorithm

In [4]:
# Definition of the matrix

theta = lambda x: np.sign(x) if x != 0 else 1

def reflection_matrix(v):
    n = v.shape[0]
    beta = 2 / np.linalg.norm(v, ord=2)**2
    
    return np.eye(n) - beta * np.dot(v, v.T) 

def householder_vector(x):
    n = x.shape[0]
    alpha = np.linalg.norm(x) * theta(x[0])
    
    return x - alpha*np.eye(n,1)

In [5]:
# Decomposition algorithm

def householder_qr(A):
    n = A.shape[0]
    m = A.shape[1]
    
    R = A.copy()
    Q_T = np.eye(n)
    
    for i in range(m):
        v = householder_vector(R[i:, i].reshape(n-i,1))
        P_temp = reflection_matrix(v)
        
        P = np.eye(n)
        P[i:, i:] = P_temp
        
        R = np.dot(P, R)
        Q_T = np.dot(P, Q_T)
        
    Q = Q_T.T
    
    return Q, R

In [6]:
A = np.random.randint(1, 20, (5, 3))
print(f'A = \n{A}\n')

Q, R = householder_qr(A)

print(f'Q = \n{Q}\n')
print(f'R = \n{R}\n')
print(f'QR = \n{np.dot(Q,R)}')

A = 
[[14 19  3]
 [11 10  3]
 [12  4 16]
 [ 4  8 10]
 [ 9  6 12]]

Q = 
[[ 0.59266726  0.58922877  0.25091678  0.14959699  0.46499084]
 [ 0.46566713 -0.00510597  0.41665056 -0.13159486 -0.76955387]
 [ 0.50800051 -0.66173353 -0.18206206  0.50477126  0.12689998]
 [ 0.1693335   0.41256226 -0.79965849  0.17323133 -0.36284399]
 [ 0.38100038 -0.2113871  -0.30140138 -0.82188832  0.20931078]]

R = 
[[ 2.36220236e+01  2.15900216e+01  1.75683509e+01]
 [-6.51909924e-15  1.05295284e+01 -7.24639059e+00]
 [-1.99087983e-15  1.30877037e-15 -1.25236923e+01]
 [-1.60211339e-15  2.07377055e-16 -3.29653657e-16]
 [-6.73595782e-15 -8.69254368e-16 -7.48137625e-16]]

QR = 
[[14. 19.  3.]
 [11. 10.  3.]
 [12.  4. 16.]
 [ 4.  8. 10.]
 [ 9.  6. 12.]]


<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">Givens rotation matrices
</h1>

This method is used to rotate a two dimensinal vector $x\in\mathbb{R}^2$ through the application of a rotation matrix 

$$G =\begin{bmatrix} \cos\theta & \sin\theta \\ -\sin\theta & \cos\theta \end{bmatrix}$$

This gives as a result 

$$Gx = \begin{bmatrix}y\\ 0\end{bmatrix} = \begin{bmatrix}||x||_2\\0\end{bmatrix}$$

provided that we defined the components of $G$ through:

$$\cos\theta = \frac{x_1}{\sqrt{x_1^2+x_2^2}},\ \ \ \ \ \sin\theta=\frac{x_2}{\sqrt{x_1^2+x_2^2}}$$

### Algorithm

In [7]:
# Definition of the matrix
def rotation_matrix(x):
    cos_theta = x[0, 0] / np.sqrt(x[0, 0]**2 + x[1, 0]**2)
    sin_theta = x[1, 0] / np.sqrt(x[0, 0]**2 + x[1, 0]**2)
    
    G = np.array([[cos_theta, sin_theta], [-sin_theta, cos_theta]])
    
    return G

In [8]:
# Decomposition algorithm

def rotation_matrix_decomposition(A):
    n = A.shape[0]
    m = A.shape[1]

    R = A.copy()
    Q_T = np.eye(n)

    for j in range(m):
        for i in range(n-1, j, -1):
            
            x = R[i-1:i+1, j]
            x = np.array([[x[0]], [x[1]]])
         
            G_temp = rotation_matrix(x)

            G = np.eye(n)
            G[i-1:i+1, i-1:i+1] = G_temp

            R = np.dot(G, R)
            Q_T = np.dot(G, Q_T)
            
    Q = Q_T.T

    return Q, R

In [9]:
A = np.random.randint(1, 20, (5, 3))
print(f'A = \n{A}\n')

Q, R = rotation_matrix_decomposition(A)

print(f'Q = \n{Q}\n')
print(f'R = \n{R}\n')
print(f'QR = \n{np.dot(Q,R)}')

A = 
[[ 7  2 15]
 [ 8  3 12]
 [ 5  4 13]
 [14  2 12]
 [14 17  6]]

Q = 
[[ 0.30406057 -0.20473835  0.52458638 -0.76839996  0.        ]
 [ 0.34749779 -0.16982436  0.23627132  0.34405865 -0.82231653]
 [ 0.21718612  0.0847427   0.66639788  0.51831198  0.48261312]
 [ 0.60812114 -0.58913122 -0.42510668  0.1073897   0.30143797]
 [ 0.60812114  0.75827764 -0.21019793 -0.10490608 -0.00390464]]

R = 
[[ 2.30217289e+01  1.40736607e+01  2.25004822e+01]
 [ 8.22671039e-16  1.11324784e+01 -6.52722137e+00]
 [-2.41523122e-16  2.21355038e-16  1.30047562e+01]
 [ 1.70499600e-17 -2.69637282e-18  4.17068090e-16]
 [ 3.63177952e-16 -2.42277666e-16 -1.13527251e-16]]

QR = 
[[ 7.  2. 15.]
 [ 8.  3. 12.]
 [ 5.  4. 13.]
 [14.  2. 12.]
 [14. 17.  6.]]
